In [1]:
import numpy as np
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [2]:
actions = np.array(['1. loud','2. quiet','3. happy','4. sad','5. Beautiful','6. Ugly','7. Deaf','8. Blind','9. Nice','10. Mean','11. rich','12. poor','13. thick','14. thin','15. expensive','16. cheap','17. flat','18. curved','19. male','20. female','21. tight','22. loose','23. high','24. low','25. soft','26. hard','27. deep','28. shallow','29. clean','30. dirty','31. strong','32. weak','33. dead','34. alive','35. heavy'])
label_map = {label: num for num, label in enumerate(actions)}

DATA_PATH = "C:\\Users\\Keerthi Mupparaju\\Desktop\\Frames3"

In [3]:
sequences = []
labels = []

for action in actions:
    action_folder = os.path.join(DATA_PATH, action)
    subfolders = [d for d in os.listdir(action_folder) if os.path.isdir(os.path.join(action_folder, d))]

    for subfolder in subfolders:
        sequence = []
        npy_files = [f for f in os.listdir(os.path.join(action_folder, subfolder)) if f.endswith('.npy')]

        for frame_num in range(len(npy_files)):
            frame_data = np.load(os.path.join(action_folder, subfolder, f"{frame_num}.npy"))
            sequence.append(frame_data)

        sequences.append(sequence)
        labels.append(label_map[action])

# Pad sequences with zero-padding
#sequences = pad_sequences(sequences, dtype='float16', padding='post')

In [4]:
X = np.array(sequences)
y = to_categorical(labels).astype(int)

In [5]:
y

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]])

In [6]:
X.shape

(622, 40, 1662)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [28]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, BatchNormalization, Dense, Dropout, TimeDistributed,LeakyReLU


# Assuming you have sequences of frames, each represented as a vector of features
# X_train shape: (num_samples, num_frames, num_features_per_frame)
# y_train shape: (num_samples, num_classes)
model = Sequential([
    #TimeDistributed(Bidirectional(LSTM(64,return_sequences=True,))),
    Bidirectional(LSTM(128, return_sequences=True,activation='tanh',input_shape=(40, 1662))),
    Dropout(0.2),  # Adjust dropout rate
    Bidirectional(LSTM(64)),
    LeakyReLU(alpha=0.1),  # Reduce LSTM units
    Dense(32, activation=LeakyReLU(alpha=0.1)),
    Dropout(0.2),  # Add another dropout layer
    Dense(52, activation=LeakyReLU(alpha=0.1)),
    Dropout(0.3),# Add another dropout layer
    Dense(42, activation=LeakyReLU(alpha=0.1)),
    Dropout(0.3),
    Dense(35, activation='softmax')
]) 


model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['CategoricalAccuracy'])

# Train the model
model.fit(X_train, y_train, epochs=2000, batch_size=64, validation_split=0.1)


Epoch 1/2000
7/7 [==============================] - 15s 997ms/step - loss: 3.5644 - categorical_accuracy: 0.0291 - val_loss: 3.5609 - val_categorical_accuracy: 0.0800
Epoch 2/2000
7/7 [==============================] - 2s 284ms/step - loss: 3.5533 - categorical_accuracy: 0.0425 - val_loss: 3.5404 - val_categorical_accuracy: 0.0800
Epoch 3/2000
7/7 [==============================] - 2s 290ms/step - loss: 3.5463 - categorical_accuracy: 0.0425 - val_loss: 3.5347 - val_categorical_accuracy: 0.0800
Epoch 4/2000
7/7 [==============================] - 2s 276ms/step - loss: 3.5505 - categorical_accuracy: 0.0492 - val_loss: 3.5372 - val_categorical_accuracy: 0.0800
Epoch 5/2000
7/7 [==============================] - 2s 317ms/step - loss: 3.5318 - categorical_accuracy: 0.0626 - val_loss: 3.5286 - val_categorical_accuracy: 0.0800
Epoch 6/2000
7/7 [==============================] - 2s 349ms/step - loss: 3.5349 - categorical_accuracy: 0.0537 - val_loss: 3.5128 - val_categorical_accuracy: 0.0800
Epo

In [30]:
# Evaluate the model on test data
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test accuracy: {accuracy}')


4/4 [==============================] - 0s 76ms/step - loss: 1.8005 - categorical_accuracy: 0.7600
Test accuracy: 0.7599999904632568


In [31]:
model.save('action75a.h5')

C:\Users\Keerthi Mupparaju\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [33]:
model.load_weights('action75a.h5')

In [34]:
model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_28 (Bidirect  (None, 40, 256)           1833984   
 ional)                                                          
                                                                 
 dropout_47 (Dropout)        (None, 40, 256)           0         
                                                                 
 bidirectional_29 (Bidirect  (None, 128)               164352    
 ional)                                                          
                                                                 
 leaky_re_lu_16 (LeakyReLU)  (None, 128)               0         
                                                                 
 dense_45 (Dense)            (None, 32)                4128      
                                                                 
 dropout_48 (Dropout)        (None, 32)              

In [35]:
res = model.predict(X_test)

4/4 [==============================] - 4s 73ms/step


In [36]:
actions[np.argmax(res[8])]

'31. strong'

In [37]:
actions[np.argmax(y_test[8])]

'31. strong'

In [33]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [ ]:
yhat = model.predict(X_test)

In [42]:
folder_path = "C:\\Users\\Keerthi Mupparaju\\Desktop\\output"  # Replace with the actual path

# Get all NumPy files (.npy) within the folder
npy_files = [f for f in os.listdir(folder_path) if f.endswith('.npy')]

# Load and append frames for a single sequence
sequence = []  # List to store frames
for frame_num in range(len(npy_files)):
  frame_data = np.load(os.path.join(folder_path, f"frame_{frame_num:05d}.npy"))
  sequence.append(frame_data)


In [50]:
single_video_input = np.expand_dims(sequence, axis=0)

single_video_input.shape

(1, 40, 1662)

In [53]:
prediction = model.predict(single_video_input)

1/1 [==============================] - 0s 65ms/step


In [59]:
predicted_class_index = np.argmax(prediction[0])
class_name = actions[predicted_class_index].split()
class_name[1]


'Blind'